# Filter tide gauges

The purpose of this notebooks is to select tide gauges and put them in the same netcdf dataset. We filter datasets according to the following criteria:
1. If a tide gauge has more than one record_id, we discard it.
2. If a tide gauge begins later than 1970, we discard it.
3. Selection of tide gauges with >20% nans? Will discard after selecting seasons

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

import datetime
import os
import glob

import attrs, utils

DATA_PATH = attrs.DATA_PATH

In [2]:
def expand_time_coord(sea_level):
    """Expands time dimension to range to maximumum timespan"""
    tmax = attrs.tmax
    tmin = attrs.tmin
    full_timespan = xr.DataArray(pd.date_range(start=tmin, end=tmax, freq='D'), dims='time')
    return sea_level.interp(time=full_timespan)

In [3]:
# 11-28-2021: Fabrizio asked for dataset for all tide gauges for all time, so we do that here:
def repeats_time(sea_level):
    """Returns true if there are ever more than one t"""
    return len(np.unique(sea_level.time)) != len(sea_level.time)

d = {}

timeseries_path = os.path.join(attrs.DATA_PATH, 'tide_gauge_timeseries')
file_list = glob.glob(os.path.join(timeseries_path, '*.nc'))
full_timespan = xr.DataArray(
    pd.date_range(start=attrs.tmin, end=attrs.tmax, freq='D')
    , dims='time'
)

for file in file_list:
    timeseries = xr.open_dataset(file)
    sea_level = timeseries.sea_level

    # Check to make sure there are no overlapping times:
    if not repeats_time(sea_level):
        sea_level = expand_time_coord(sea_level)
        
        # Add ds attributes to dataarray
        sea_level.attrs['latitude'] = timeseries.latitude[0].item()
        sea_level.attrs['longitude'] = timeseries.longitude[0].item()
        sea_level.attrs['first_record_date'] = timeseries.first_record_date
        sea_level.attrs['last_record_date'] = timeseries.last_record_date
        # sea_level.attrs['num_days'] = timeseries.num_days
        
        # Add to dictionary
        station_name = utils.string_regex(timeseries.station_name)
        d[station_name] = sea_level

ds = xr.Dataset(d)
ds.to_netcdf(os.path.join(DATA_PATH, 'tide_gauges_all.nc'))

In [4]:
ds

<xarray.Dataset>
Dimensions:             (time: 63394)
Coordinates:
  * time                (time) datetime64[ns] 1846-01-04T12:00:00 ... 2019-07...
Data variables:
    corinto             (time) float64 nan nan nan nan nan ... nan nan nan nan
    rorvik              (time) float64 nan nan nan nan nan ... nan nan nan nan
    lusi                (time) float64 nan nan nan nan nan ... nan nan nan nan
    port_moresby        (time) float64 nan nan nan nan nan ... nan nan nan nan
    marseille           (time) float64 nan nan nan nan nan ... nan nan nan nan
    easter              (time) float64 nan nan nan nan nan ... nan nan nan nan
    bintulu             (time) float64 nan nan nan nan nan ... nan nan nan nan
    port_allen          (time) float64 nan nan nan nan nan ... nan nan nan nan
    le_robert           (time) float64 nan nan nan nan nan ... nan nan nan nan
    noumea              (time) float64 nan nan nan nan nan ... nan nan nan nan
    puerto_madryn       (time) float64 nan nan nan nan nan ... nan nan nan nan
    cayenne             (time) float64 nan nan nan nan nan ... nan nan nan nan
    yap                 (time) float64 nan nan nan nan nan ... nan nan nan nan
    manta               (time) float64 nan nan nan nan nan ... nan nan nan nan
    cape_town           (time) float64 nan nan nan nan nan ... nan nan nan nan
    roseau              (time) float64 nan nan nan nan nan ... nan nan nan nan
    midway              (time) float64 nan nan nan nan nan ... nan nan nan nan
    chittagong          (time) float64 nan nan nan nan nan ... nan nan nan nan
    lord_howe           (time) float64 nan nan nan nan nan ... nan nan nan nan
    lautoka             (time) float64 nan nan nan nan nan ... nan nan nan nan
    nuku_hiva           (time) float64 nan nan nan nan nan ... nan nan nan nan
    dutch_harbor_ak     (time) float64 nan nan nan nan nan ... nan nan nan nan
    south_pass_la       (time) float64 nan nan nan nan nan ... nan nan nan nan
    kaumalapau_hi       (time) float64 nan nan nan nan nan ... nan nan nan nan
    thevenard           (time) float64 nan nan nan nan nan ... nan nan nan nan
    crozet              (time) float64 nan nan nan nan nan ... nan nan nan nan
    magueyes_island_pr  (time) float64 nan nan nan nan nan ... nan nan nan nan
    port_louis          (time) float64 nan nan nan nan nan ... nan nan nan nan
    pago_bay_guam       (time) float64 nan nan nan nan nan ... nan nan nan nan
    luanda              (time) float64 nan nan nan nan nan ... nan nan nan nan
    exuma_cays          (time) float64 nan nan nan nan nan ... nan nan nan nan
    charchanga          (time) float64 nan nan nan nan nan ... nan nan nan nan
    esperanza           (time) float64 nan nan nan nan nan ... nan nan nan nan
    providenya          (time) float64 nan nan nan nan nan ... nan nan nan nan
    tawau               (time) float64 nan nan nan nan nan ... nan nan nan nan
    panama_city_beach   (time) float64 nan nan nan nan nan ... nan nan nan nan
    churchill           (time) float64 nan nan nan nan nan ... nan nan nan nan
    mazatlan            (time) float64 nan nan nan nan nan ... nan nan nan nan
    muscat              (time) float64 nan nan nan nan nan ... nan nan nan nan
    esmeraldas          (time) float64 nan nan nan nan nan ... nan nan nan nan
    auckland            (time) float64 nan nan nan nan nan ... nan nan nan nan
    imbituba            (time) float64 nan nan nan nan nan ... nan nan nan nan
    bluff               (time) float64 nan nan nan nan nan ... nan nan nan nan
    puerto_plata        (time) float64 nan nan nan nan nan ... nan nan nan nan
    ile_royale          (time) float64 nan nan nan nan nan ... nan nan nan nan
    port_san_luis_ca    (time) float64 nan nan nan nan nan ... nan nan nan nan
    mombasa             (time) float64 nan nan nan nan nan ... nan nan nan nan
    san_francisco_ca    (time) float64 nan nan nan nan nan ... nan nan nan nan
    puerto_williams     (time) float64 nan 

In [4]:
start_date_threshold = '1970-01-01'
end_date_threshold = '2018-01-01'
end_date_threshold_m1 = '2017-12-31'

def conditions(timeseries):
    """
    Returns true if the specified conditions are true for the timeseries.
    """
    return (
        (timeseries.num_record_changes == 0) and
        (np.datetime64(timeseries.first_record_date) <= np.datetime64(start_date_threshold)) and
        (np.datetime64(end_date_threshold_m1) <= np.datetime64(timeseries.last_record_date))
    )

In [5]:
# Loop
d = {}

timeseries_path = os.path.join(attrs.DATA_PATH, 'tide_gauge_timeseries')
file_list = glob.glob(os.path.join(timeseries_path, '*.nc'))

for file in file_list:
    timeseries = xr.open_dataset(file)
    if conditions(timeseries):
        sea_level = timeseries.sea_level
        sea_level = expand_time_coord(sea_level)
        
        # Add ds attributes to dataarray
        sea_level.attrs['latitude'] = timeseries.latitude[0].item()
        sea_level.attrs['longitude'] = timeseries.longitude[0].item()
        sea_level.attrs['first_record_date'] = timeseries.first_record_date
        sea_level.attrs['last_record_date'] = timeseries.last_record_date
        # sea_level.attrs['num_days'] = timeseries.num_days
        
        # Add to dictionary
        station_name = utils.string_regex(timeseries.station_name)
        d[station_name] = sea_level

ds = xr.Dataset(d)
ds

<xarray.Dataset>
Dimensions:             (time: 63394)
Coordinates:
  * time                (time) datetime64[ns] 1846-01-04T12:00:00 ... 2019-07...
Data variables:
    rorvik              (time) float64 nan nan nan nan nan ... nan nan nan nan
    marseille           (time) float64 nan nan nan nan nan ... nan nan nan nan
    noumea              (time) float64 nan nan nan nan nan ... nan nan nan nan
    midway              (time) float64 nan nan nan nan nan ... nan nan nan nan
    magueyes_island_pr  (time) float64 nan nan nan nan nan ... nan nan nan nan
    churchill           (time) float64 nan nan nan nan nan ... nan nan nan nan
    port_san_luis_ca    (time) float64 nan nan nan nan nan ... nan nan nan nan
    san_francisco_ca    (time) float64 nan nan nan nan nan ... nan nan nan nan
    sitka_ak            (time) float64 nan nan nan nan nan ... nan nan nan nan
    nain                (time) float64 nan nan nan nan nan ... nan nan nan nan
    san_diego           (time) float64 nan nan nan nan nan ... nan nan nan nan
    valparaiso          (time) float64 nan nan nan nan nan ... nan nan nan nan
    newport_ri          (time) float64 nan nan nan nan nan ... nan nan nan nan
    dzaoudzi            (time) float64 nan nan nan nan nan ... nan nan nan nan
    portland_me         (time) float64 nan nan nan nan nan ... nan nan nan nan
    la_coruna           (time) float64 nan nan nan nan nan ... nan nan nan nan
    johnston            (time) float64 nan nan nan nan nan ... nan nan nan nan
    ofunato             (time) float64 nan nan nan nan nan ... nan nan nan nan
    abashiri            (time) float64 nan nan nan nan nan ... nan nan nan nan
    taranaki            (time) float64 nan nan nan nan nan ... nan nan nan nan
    pago_pago           (time) float64 nan nan nan nan nan ... nan nan nan nan
    simons_town         (time) float64 nan nan nan nan nan ... nan nan nan nan
    wilmington_nc       (time) float64 nan nan nan nan nan ... nan nan nan nan
    cuxhaven            (time) float64 nan nan nan nan nan ... nan nan nan nan
    kwajalein           (time) float64 nan nan nan nan nan ... nan nan nan nan
    naha                (time) float64 nan nan nan nan nan ... nan nan nan nan
    pensacola_fl        (time) float64 nan nan nan nan nan ... nan nan nan nan
    wake                (time) float64 nan nan nan nan nan ... nan nan nan nan
    new_london_ct       (time) float64 nan nan nan nan nan ... nan nan nan nan
    rikitea             (time) float64 nan nan nan nan nan ... nan nan nan nan
    mera                (time) float64 nan nan nan nan nan ... nan nan nan nan
    nishinoomote        (time) float64 nan nan nan nan nan ... nan nan nan nan
    kahului             (time) float64 nan nan nan nan nan ... nan nan nan nan
    naze                (time) float64 nan nan nan nan nan ... nan nan nan nan
    cristobal           (time) float64 nan nan nan nan nan ... nan nan nan nan
    tofino              (time) float64 nan nan nan nan nan ... nan nan nan nan
    fernandina_beach    (time) float64 nan nan nan nan nan ... nan nan nan nan
    vardo               (time) float64 nan nan nan nan nan ... nan nan nan nan
    woods_hole_ma       (time) float64 nan nan nan nan nan ... nan nan nan nan
    kushimoto           (time) float64 nan nan nan nan nan ... nan nan nan nan
    ketchikan_ak        (time) float64 nan nan nan nan nan ... nan nan nan nan
    cape_may_nj         (time) float64 nan nan nan nan nan ... nan nan nan nan
    montauk_ny          (time) float64 nan nan nan nan nan ... nan nan nan nan
    victoria_bc         (time) float64 nan nan nan nan nan ... nan nan nan nan
    naples_fl           (time) float64 nan nan nan nan nan ... nan nan nan nan
    wellington          (time) float64 nan nan nan nan nan ... nan nan nan nan
    miyakejima          (time) float64 nan nan nan nan nan ... nan nan nan nan
    nawiliwili          (time) float64 nan nan nan nan nan ... nan nan nan nan
    mokuoloe            (time) float64 nan 

In [6]:
# Select from 1970 onwards
ds_1970_2017 = ds.sel(time=slice(start_date_threshold, end_date_threshold))
ds_1970_2017

<xarray.Dataset>
Dimensions:             (time: 17533)
Coordinates:
  * time                (time) datetime64[ns] 1970-01-01T12:00:00 ... 2018-01...
Data variables:
    rorvik              (time) float64 1.359e+03 1.369e+03 ... 1.508e+03
    marseille           (time) float64 653.0 573.0 621.0 ... 446.0 497.0 547.0
    noumea              (time) float64 1.152e+03 1.103e+03 ... 1.115e+03
    midway              (time) float64 1.145e+03 1.105e+03 ... 1.172e+03
    magueyes_island_pr  (time) float64 1.106e+03 1.106e+03 1.124e+03 ... nan nan
    churchill           (time) float64 2.929e+03 2.929e+03 ... 2.524e+03
    port_san_luis_ca    (time) float64 2.178e+03 2.188e+03 ... 2.204e+03
    san_francisco_ca    (time) float64 2.737e+03 2.732e+03 ... 2.809e+03
    sitka_ak            (time) float64 3.024e+03 2.944e+03 ... 2.885e+03
    nain                (time) float64 1.646e+03 1.597e+03 ... 1.397e+03
    san_diego           (time) float64 1.886e+03 1.879e+03 ... 2.145e+03
    valparaiso          (time) float64 888.0 885.0 868.0 ... 787.0 786.0 769.0
    newport_ri          (time) float64 1.064e+03 964.0 975.0 ... 1.075e+03 977.0
    dzaoudzi            (time) float64 nan nan nan nan nan ... nan nan nan nan
    portland_me         (time) float64 4.089e+03 4.05e+03 ... 4.048e+03
    la_coruna           (time) float64 2.734e+03 2.736e+03 ... 2.76e+03 nan
    johnston            (time) float64 944.0 932.0 940.0 ... 765.0 778.0 778.0
    ofunato             (time) float64 2.634e+03 2.607e+03 ... 2.52e+03
    abashiri            (time) float64 1.409e+03 1.408e+03 ... 1.556e+03
    taranaki            (time) float64 nan nan nan ... 2.067e+03 2.1e+03
    pago_pago           (time) float64 1.153e+03 1.144e+03 ... 1.463e+03
    simons_town         (time) float64 1.071e+03 1.04e+03 ... nan 1.171e+03
    wilmington_nc       (time) float64 1.247e+03 1.331e+03 ... 1.399e+03
    cuxhaven            (time) float64 4.884e+03 5.402e+03 ... 5.646e+03
    kwajalein           (time) float64 nan nan nan ... 1.576e+03 1.586e+03
    naha                (time) float64 2.539e+03 2.513e+03 ... 2.423e+03
    pensacola_fl        (time) float64 2.538e+03 2.512e+03 ... 2.581e+03
    wake                (time) float64 1.57e+03 1.569e+03 ... 1.766e+03
    new_london_ct       (time) float64 1.498e+03 1.378e+03 ... 1.417e+03
    rikitea             (time) float64 924.0 924.0 904.0 ... 973.0 969.0 968.0
    mera                (time) float64 1.334e+03 1.302e+03 ... 1.379e+03
    nishinoomote        (time) float64 2.213e+03 2.205e+03 ... 2.247e+03
    kahului             (time) float64 1.063e+03 1.072e+03 ... 1.182e+03
    naze                (time) float64 2.341e+03 2.319e+03 ... 2.299e+03
    cristobal           (time) float64 657.0 673.0 669.0 ... 758.0 753.0 nan
    tofino              (time) float64 1.979e+03 1.966e+03 ... 1.994e+03
    fernandina_beach    (time) float64 nan nan nan ... 1.43e+03 1.722e+03
    vardo               (time) float64 nan nan nan ... 2.934e+03 2.934e+03
    woods_hole_ma       (time) float64 1.057e+03 964.0 ... 1.118e+03 1.026e+03
    kushimoto           (time) float64 1.697e+03 1.666e+03 ... 1.705e+03
    ketchikan_ak        (time) float64 4.268e+03 4.251e+03 ... 4.238e+03
    cape_may_nj         (time) float64 nan nan nan ... 1.482e+03 1.425e+03
    montauk_ny          (time) float64 1.508e+03 1.381e+03 ... 1.488e+03
    victoria_bc         (time) float64 1.687e+03 1.713e+03 ... 1.796e+03
    naples_fl           (time) float64 1.054e+03 952.0 ... 1.145e+03 1.141e+03
    wellington          (time) float64 996.0 1.01e+03 ... 1.205e+03 1.246e+03
    miyakejima          (time) float64 nan nan nan ... 2.655e+03 2.591e+03
    nawiliwili          (time) float64 937.0 938.0 922.0 ... 1.006e+03 999.0
    mokuoloe            (time) float64 nan nan nan ... 1.286e+03 1.289e+03
    brest               (time) float64 4.106e+03 4.115e+03 ... 4.305e+03
    nantucket_ma        (time) float64 1.445e+03 1.348e+03 ... 1.457e+03
    tregde              (ti

In [6]:
# Drop any variables with only nans
ds_nonan = xr.Dataset({
    var_name: var for var_name, var in ds_1970_2017.items() if (
        ~np.all(np.isnan(var))
    )
})

ds_nonan
# Save ds and variable names
ds_nonan.to_netcdf(os.path.join(DATA_PATH, 'tide_gauges_1970-2017.nc'))
np.save(os.path.join(DATA_PATH, 'var_names_1970-2017.npy'), list(ds_nonan.keys()))

In [7]:
# Select JJASON
def is_jjason(month):
    return (month >= 6) & (month <= 11)
ds_JJASON = ds_nonan.sel(time=is_jjason(ds_nonan['time.month']))

ds_JJASON

<xarray.Dataset>
Dimensions:             (time: 8784)
Coordinates:
  * time                (time) datetime64[ns] 1970-06-01T12:00:00 ... 2017-11...
Data variables:
    rorvik              (time) float64 1.2e+03 1.147e+03 ... 1.316e+03 1.237e+03
    marseille           (time) float64 362.0 388.0 416.0 ... 547.0 623.0 592.0
    noumea              (time) float64 1.022e+03 1.038e+03 ... 1.04e+03
    midway              (time) float64 1.006e+03 1.004e+03 ... 1.1e+03 1.116e+03
    magueyes_island_pr  (time) float64 1.088e+03 1.073e+03 1.063e+03 ... nan nan
    churchill           (time) float64 2.874e+03 2.815e+03 ... 2.65e+03
    port_san_luis_ca    (time) float64 2.042e+03 2.055e+03 ... 2.224e+03
    san_francisco_ca    (time) float64 2.664e+03 2.66e+03 ... 2.772e+03
    sitka_ak            (time) float64 2.969e+03 3.044e+03 ... 3.031e+03
    nain                (time) float64 1.327e+03 1.336e+03 ... 1.444e+03
    san_diego           (time) float64 1.91e+03 1.918e+03 ... 2.159e+03
    valparaiso          (time) float64 886.0 911.0 878.0 ... 806.0 730.0 758.0
    newport_ri          (time) float64 937.0 1.007e+03 ... 1.132e+03 1.156e+03
    dzaoudzi            (time) float64 nan nan nan nan nan ... nan nan nan nan
    portland_me         (time) float64 4.083e+03 4.13e+03 ... 4.106e+03
    la_coruna           (time) float64 2.598e+03 2.653e+03 ... 2.713e+03
    johnston            (time) float64 970.0 956.0 942.0 ... 935.0 932.0 966.0
    ofunato             (time) float64 2.698e+03 2.626e+03 ... 2.46e+03
    abashiri            (time) float64 1.366e+03 1.331e+03 ... 1.457e+03
    taranaki            (time) float64 nan nan nan ... 1.938e+03 1.933e+03
    pago_pago           (time) float64 1.239e+03 1.209e+03 ... 1.506e+03
    simons_town         (time) float64 843.0 913.0 1.018e+03 ... nan nan nan
    wilmington_nc       (time) float64 1.47e+03 1.379e+03 ... 1.576e+03
    cuxhaven            (time) float64 5.344e+03 5.127e+03 ... 5.27e+03
    kwajalein           (time) float64 1.528e+03 1.512e+03 ... 1.6e+03 1.601e+03
    naha                (time) float64 2.461e+03 2.449e+03 ... 2.566e+03
    pensacola_fl        (time) float64 2.951e+03 2.957e+03 ... 2.868e+03
    wake                (time) float64 1.592e+03 1.593e+03 ... 1.76e+03
    new_london_ct       (time) float64 1.373e+03 1.449e+03 ... 1.596e+03
    rikitea             (time) float64 756.0 763.0 779.0 ... 875.0 875.0 889.0
    mera                (time) float64 1.319e+03 1.266e+03 ... 1.388e+03
    nishinoomote        (time) float64 2.416e+03 2.343e+03 ... 2.256e+03
    kahului             (time) float64 934.0 939.0 939.0 ... 1.199e+03 1.229e+03
    naze                (time) float64 2.351e+03 2.33e+03 ... 2.444e+03
    cristobal           (time) float64 737.0 732.0 738.0 ... 923.0 906.0 912.0
    tofino              (time) float64 1.965e+03 2.051e+03 ... 2.23e+03
    fernandina_beach    (time) float64 nan nan nan ... 1.626e+03 1.628e+03
    vardo               (time) float64 nan nan nan ... 2.986e+03 2.897e+03
    woods_hole_ma       (time) float64 977.0 1.054e+03 ... 1.145e+03 1.172e+03
    kushimoto           (time) float64 1.892e+03 1.818e+03 ... 1.765e+03
    ketchikan_ak        (time) float64 4.28e+03 4.351e+03 ... 4.512e+03 4.45e+03
    cape_may_nj         (time) float64 nan nan nan ... 1.585e+03 1.607e+03
    montauk_ny          (time) float64 1.361e+03 1.436e+03 ... 1.636e+03
    victoria_bc         (time) float64 1.723e+03 1.813e+03 ... 1.979e+03
    naples_fl           (time) float64 1.165e+03 1.144e+03 ... 1.287e+03
    wellington          (time) float64 946.0 1.088e+03 ... 1.038e+03 1.028e+03
    miyakejima          (time) float64 2.305e+03 2.221e+03 ... 2.828e+03
    nawiliwili          (time) float64 842.0 833.0 834.0 ... 1.038e+03 1.06e+03
    mokuoloe            (time) float64 nan nan nan ... 1.317e+03 1.339e+03
    brest               (time) float64 3.919e+03 3.916e+03 ... 4.163e+03
    nantucket_ma        (time) float64 1.276e+03 1.321e+03 ... 1.531e+03
 

In [11]:
# Remove any datasets with more than 20% nans

def percent_nan(sea_level):
    return np.sum(np.isnan(sea_level)).item() / len(sea_level)

nan_threshold = 0.2

ds_filtered = xr.Dataset({
    var_name: var for var_name, var in ds_JJASON.items() if (
        percent_nan(var) <= nan_threshold
    )
})
ds_filtered

<xarray.Dataset>
Dimensions:             (time: 8784)
Coordinates:
  * time                (time) datetime64[ns] 1970-06-01T12:00:00 ... 2017-11...
Data variables:
    rorvik              (time) float64 1.2e+03 1.147e+03 ... 1.316e+03 1.237e+03
    noumea              (time) float64 1.022e+03 1.038e+03 ... 1.04e+03
    midway              (time) float64 1.006e+03 1.004e+03 ... 1.1e+03 1.116e+03
    magueyes_island_pr  (time) float64 1.088e+03 1.073e+03 1.063e+03 ... nan nan
    churchill           (time) float64 2.874e+03 2.815e+03 ... 2.65e+03
    port_san_luis_ca    (time) float64 2.042e+03 2.055e+03 ... 2.224e+03
    san_francisco_ca    (time) float64 2.664e+03 2.66e+03 ... 2.772e+03
    sitka_ak            (time) float64 2.969e+03 3.044e+03 ... 3.031e+03
    san_diego           (time) float64 1.91e+03 1.918e+03 ... 2.159e+03
    newport_ri          (time) float64 937.0 1.007e+03 ... 1.132e+03 1.156e+03
    portland_me         (time) float64 4.083e+03 4.13e+03 ... 4.106e+03
    la_coruna           (time) float64 2.598e+03 2.653e+03 ... 2.713e+03
    johnston            (time) float64 970.0 956.0 942.0 ... 935.0 932.0 966.0
    ofunato             (time) float64 2.698e+03 2.626e+03 ... 2.46e+03
    abashiri            (time) float64 1.366e+03 1.331e+03 ... 1.457e+03
    taranaki            (time) float64 nan nan nan ... 1.938e+03 1.933e+03
    pago_pago           (time) float64 1.239e+03 1.209e+03 ... 1.506e+03
    wilmington_nc       (time) float64 1.47e+03 1.379e+03 ... 1.576e+03
    cuxhaven            (time) float64 5.344e+03 5.127e+03 ... 5.27e+03
    kwajalein           (time) float64 1.528e+03 1.512e+03 ... 1.6e+03 1.601e+03
    naha                (time) float64 2.461e+03 2.449e+03 ... 2.566e+03
    pensacola_fl        (time) float64 2.951e+03 2.957e+03 ... 2.868e+03
    wake                (time) float64 1.592e+03 1.593e+03 ... 1.76e+03
    new_london_ct       (time) float64 1.373e+03 1.449e+03 ... 1.596e+03
    rikitea             (time) float64 756.0 763.0 779.0 ... 875.0 875.0 889.0
    mera                (time) float64 1.319e+03 1.266e+03 ... 1.388e+03
    nishinoomote        (time) float64 2.416e+03 2.343e+03 ... 2.256e+03
    kahului             (time) float64 934.0 939.0 939.0 ... 1.199e+03 1.229e+03
    naze                (time) float64 2.351e+03 2.33e+03 ... 2.444e+03
    cristobal           (time) float64 737.0 732.0 738.0 ... 923.0 906.0 912.0
    tofino              (time) float64 1.965e+03 2.051e+03 ... 2.23e+03
    woods_hole_ma       (time) float64 977.0 1.054e+03 ... 1.145e+03 1.172e+03
    kushimoto           (time) float64 1.892e+03 1.818e+03 ... 1.765e+03
    ketchikan_ak        (time) float64 4.28e+03 4.351e+03 ... 4.512e+03 4.45e+03
    cape_may_nj         (time) float64 nan nan nan ... 1.585e+03 1.607e+03
    montauk_ny          (time) float64 1.361e+03 1.436e+03 ... 1.636e+03
    victoria_bc         (time) float64 1.723e+03 1.813e+03 ... 1.979e+03
    naples_fl           (time) float64 1.165e+03 1.144e+03 ... 1.287e+03
    wellington          (time) float64 946.0 1.088e+03 ... 1.038e+03 1.028e+03
    miyakejima          (time) float64 2.305e+03 2.221e+03 ... 2.828e+03
    nawiliwili          (time) float64 842.0 833.0 834.0 ... 1.038e+03 1.06e+03
    brest               (time) float64 3.919e+03 3.916e+03 ... 4.163e+03
    nantucket_ma        (time) float64 1.276e+03 1.321e+03 ... 1.531e+03
    tregde              (time) float64 949.0 873.0 777.0 ... 1.002e+03 980.0
    toyama              (time) float64 1.205e+03 1.138e+03 ... 1.255e+03
    kushiro             (time) float64 1.574e+03 1.582e+03 ... 1.82e+03
    key_west_fl         (time) float64 1.613e+03 1.579e+03 ... 1.876e+03
    aburatsu            (time) float64 1.549e+03 1.54e+03 ... 1.426e+03
    boston_ma           (time) float64 2.53e+03 2.569e+03 ... 2.699e+03
    adak_alaska         (time) float64 1.323e+03 1.459e+03 ... 1.521e+03
    la_jolla_ca         (time) float64 2.061e+03 2.083e+03 ... 2.2e+03 2.203e+03
    tumaco          

In [12]:
ds_filtered.to_netcdf(os.path.join(DATA_PATH, 'tide_gauges_filtered.nc'))
np.save(os.path.join(DATA_PATH, 'var_names_filtered.npy'), list(ds_nonan.keys()))

In [14]:
ds = xr.open_dataset(os.path.join(DATA_PATH, 'tide_gauges_filtered.nc'))
ds

<xarray.Dataset>
Dimensions:             (time: 8784)
Coordinates:
  * time                (time) datetime64[ns] 1970-06-01T12:00:00 ... 2017-11...
Data variables:
    rorvik              (time) float64 ...
    noumea              (time) float64 ...
    midway              (time) float64 ...
    magueyes_island_pr  (time) float64 ...
    churchill           (time) float64 ...
    port_san_luis_ca    (time) float64 ...
    san_francisco_ca    (time) float64 ...
    sitka_ak            (time) float64 ...
    san_diego           (time) float64 ...
    newport_ri          (time) float64 ...
    portland_me         (time) float64 ...
    la_coruna           (time) float64 ...
    johnston            (time) float64 ...
    ofunato             (time) float64 ...
    abashiri            (time) float64 ...
    taranaki            (time) float64 ...
    pago_pago           (time) float64 ...
    wilmington_nc       (time) float64 ...
    cuxhaven            (time) float64 ...
    kwajalein           (time) float64 ...
    naha                (time) float64 ...
    pensacola_fl        (time) float64 ...
    wake                (time) float64 ...
    new_london_ct       (time) float64 ...
    rikitea             (time) float64 ...
    mera                (time) float64 ...
    nishinoomote        (time) float64 ...
    kahului             (time) float64 ...
    naze                (time) float64 ...
    cristobal           (time) float64 ...
    tofino              (time) float64 ...
    woods_hole_ma       (time) float64 ...
    kushimoto           (time) float64 ...
    ketchikan_ak        (time) float64 ...
    cape_may_nj         (time) float64 ...
    montauk_ny          (time) float64 ...
    victoria_bc         (time) float64 ...
    naples_fl           (time) float64 ...
    wellington          (time) float64 ...
    miyakejima          (time) float64 ...
    nawiliwili          (time) float64 ...
    brest               (time) float64 ...
    nantucket_ma        (time) float64 ...
    tregde              (time) float64 ...
    toyama              (time) float64 ...
    kushiro             (time) float64 ...
    key_west_fl         (time) float64 ...
    aburatsu            (time) float64 ...
    boston_ma           (time) float64 ...
    adak_alaska         (time) float64 ...
    la_jolla_ca         (time) float64 ...
    tumaco              (time) float64 ...
    guam                (time) float64 ...
    lewes_de            (time) float64 ...
    crescent_city_ca    (time) float64 ...
    antofagasta         (time) float64 ...
    fort_pulaski_ga     (time) float64 ...
    maisaka             (time) float64 ...
    ishigaki            (time) float64 ...
    rockport_tx         (time) float64 ...
    fort_denison        (time) float64 ...
    hakodate            (time) float64 ...
    south_beach_or      (time) float64 ...
    wakkanai            (time) float64 ...
    ceuta               (time) float64 ...
    new_york_ny         (time) float64 ...
    prince_rupert       (time) float64 ...
    port_isabel_tx      (time) float64 ...
    yakutat_ak          (time) float64 ...
    eastport_me         (time) float64 ...
    atlantic_city       (time) float64 ...
    neah_bay_wa         (time) float64 ...
    st_petersburg_fl    (time) float64 ...
    balboa              (time) float64 ...
    hilo_hawaii         (time) float64 ...
    astoria_or          (time) float64 ...
    galveston_pier_21   (time) float64 ...
    los_angeles_ca      (time) float64 ...
    la_libertad         (time) float64 ...